In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from loadData import *
from ML_util import *
from Dataset import IdentiGazeDataset

In [ ]:
path = 'Similar_All.csv'

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
learning_rate = 0.001
training_epochs = 400
batch_size = 32
train_vaild_Dataset = IdentiGazeDataset(path, 'train')
testDataset = IdentiGazeDataset(path, 'test')

In [4]:
train_size = int(len(train_vaild_Dataset)*0.8)
valid_size = len(train_vaild_Dataset) - train_size
train_dataset, valid_dataset = random_split(train_vaild_Dataset, [train_size, valid_size])

test_size = len(testDataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testDataset, batch_size=batch_size, shuffle=True)

<H1> Example Network

In [5]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(CausalConv1d, self).__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=self.padding, dilation=dilation)

    def forward(self, x):
        x = self.conv1d(x)
        return x[:, :, :-self.padding]  # Remove the padding on the right

class GatedActivationUnit(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(GatedActivationUnit, self).__init__()
        self.conv = CausalConv1d(in_channels, out_channels * 2, kernel_size, stride, dilation)

    def forward(self, x):
        x = self.conv(x)
        out, gate = x.chunk(2, dim=1)  # Split the tensor along the channel dimension
        return torch.tanh(out) * torch.sigmoid(gate)  # Gated activation


In [6]:
class IdentiGazeNet(nn.Module):
    def __init__(self):
        super(IdentiGazeNet, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 128)
        # Define the output layer
        self.output = nn.Linear(128, 34)

        self.RawGazeNet = nn.Sequential(
            # Input Size: (84,2)
            CausalConv1d(2, 4, kernel_size=7, stride=1, dilation=2),
            nn.MaxPool1d(kernel_size=3, stride=3),
            CausalConv1d(4, 5, kernel_size=5, stride=1, dilation=4),
            CausalConv1d(5, 16, kernel_size=3, stride=1, dilation=5),
            CausalConv1d(16, 32, kernel_size=3, stride=1, dilation=16),
            nn.AdaptiveMaxPool1d(1),  # Global Max Pooling
        )

        self.EyeMovementNet = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.FixationNet = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.SaccadeNet = nn.Sequential(
            nn.Linear(17, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.MFCCNet = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.PupilNet = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )


    def forward(self, rawgaze, eyemovement, fixation, saccade, mfcc, pupil):
        # Apply the gated convolutions
        rawgaze = self.RawGazeNet(rawgaze)
        rawgaze = rawgaze.view(rawgaze.size(0), -1)
        rawgaze = F.relu(self.fc1(rawgaze))
        rawgaze = self.dropout(rawgaze)
        rawgaze = F.relu(self.fc2(rawgaze))
        rawgaze = self.output(rawgaze)

        eyemovement = eyemovement.view(eyemovement.size(0), -1)
        eyemovement = self.EyeMovementNet(eyemovement)
        fixation = self.FixationNet(fixation)
        saccade = self.SaccadeNet(saccade)
        mfcc = self.MFCCNet(mfcc)
        pupil = self.PupilNet(pupil)

        out = rawgaze + eyemovement + fixation + saccade + mfcc + pupil
        return out


In [8]:
# writer = SummaryWriter('/runs')
idModel = IdentiGazeNet()
output = idModel(torch.randn(1,2,84).to(device), torch.randn(1,9).to(device),torch.randn(1,10).to(device),torch.randn(1,17).to(device),torch.randn(1,12).to(device),torch.randn(1,12).to(device))
# print(len(myIdentiGaze))
print(output.shape)

torch.Size([1, 34])


In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(idModel.parameters(), lr=learning_rate, weight_decay=1e-5)

In [10]:
for epoch in range(training_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader):
        rawgaze = data['rawgaze'].to(device).type(dtype=torch.float32)
        eyemovement = data['eyemovement'].to(device).type(dtype=torch.float32)
        fixation = data['fixation'].to(device).type(dtype=torch.float32)
        saccade = data['saccade'].to(device).type(dtype=torch.float32)
        mfcc = data['mfcc'].to(device).type(dtype=torch.float32)
        pupil = data['pupil'].to(device).type(dtype=torch.float32)
        y = data['y'].to(device)
        
        optimizer.zero_grad()
        outputs = idModel(rawgaze, eyemovement, fixation, saccade, mfcc, pupil)
        loss = criterion(outputs, y)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    Accuracy = 100 * correct // total
    # Tensorboard 사용하려면 아래 주석 해제
    # writer.add_scalar('Acc/train', Accuracy, epoch+1)
    # writer.add_scalar('Loss/train', running_loss, epoch+1)
    running_loss = 0.0

    
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valid_loader:
            rawgaze = data['rawgaze'].to(device).type(dtype=torch.float32)
            eyemovement = data['eyemovement'].to(device).type(dtype=torch.float32)
            fixation = data['fixation'].to(device).type(dtype=torch.float32)
            saccade = data['saccade'].to(device).type(dtype=torch.float32)
            mfcc = data['mfcc'].to(device).type(dtype=torch.float32)
            pupil = data['pupil'].to(device).type(dtype=torch.float32)
            y = data['y'].to(device)
            # calculate outputs by running images through the network
            outputs = idModel(rawgaze, eyemovement, fixation, saccade, mfcc, pupil)
            test_loss = criterion(outputs, y)

            # Tensorboard 사용하려면 아래 주석 해제
            # writer.add_scalar('Loss/valid', test_loss, epoch+1)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        Accuracy = 100 * correct // total
        print(f'Accuracy of the network on the 10000 test images: {Accuracy} %')
        # Tensorboard 사용하려면 아래 주석 해제
        # writer.add_scalar('Acc/valid', Accuracy, epoch+1)

        # 모델 저장하고싶으면 아래 주석 해제
        # torch.save(idModel.state_dict(), f'model/IdNet3/{epoch}_{100 * correct // total}.pth')


c:\Users\scilab\Desktop\논문\Identigaze Implicit reference\identigaze2\python\Dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  eyemovement = torch.FloatTensor(self.eyemovement.iloc[index])
c:\Users\scilab\Desktop\논문\Identigaze Implicit reference\identigaze2\python\Dataset.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fixation = torch.FloatTensor(self.fixation.iloc[index])
c:\Users\scilab\Desktop\논문\Identigaze Implicit reference\identigaze2\python\Dataset.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as label

Accuracy of the network on the 10000 test images: 11 %


KeyboardInterrupt: 